In [2]:
import os
import random

import numpy as np
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.algorithms.soo.nonconvex.es import ES
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.algorithms.soo.nonconvex.nelder import NelderMead
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch
from pymoo.operators.sampling.lhs import LHS
from pymoo.problems import get_problem
from pymoo.optimize import minimize
import pandas as pd
from pymoo.termination.default import DefaultSingleObjectiveTermination
from pymoo.termination.max_gen import MaximumGenerationTermination

import sys


def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    # tf.random.set_seed(seed)

def merge_generated_files(generated_trajectory_files, result_file):
    all_populations_df = pd.DataFrame()
    for file in generated_trajectory_files:
        df = pd.read_csv(file, index_col=[0], compression='zip')
        all_populations_df = pd.concat([all_populations_df, df])
    all_populations_df.to_csv(result_file, compression='zip')
    for file in generated_trajectory_files:
        os.remove(file)




def run(seed, dimension, instance_count=1000):

    set_random_seed(seed)
    maximum_generations=50
    population_size = 10*dimension
    sampling = LHS()
    ga = GA(
        pop_size=population_size,
        sampling=sampling)

    de = DE(
        pop_size=population_size,seed=seed
    )

    n_offsprings = max(population_size*2, 200)
    es = ES( pop_size=population_size, seed=seed, sampling=sampling, n_offsprings=n_offsprings)
    nm = NelderMead()
    ps = PatternSearch()
    pso = PSO(pop_size=population_size, sampling=sampling)
    cmaes=CMAES(popsize=population_size)
    algorithms = {'ES':es,'DE':de,'CMAES':cmaes, 'PSO':pso}
    #algorithms = init_de_configurations(dimension, seed)
    algorithms_to_run = [('PSO',pso),('DE',de),('ES',es)]
    result_dir = f'results_100_instances'
    os.makedirs(result_dir, exist_ok=True)

    for algorithm_name, algorithm in algorithms_to_run :
        generated_trajectory_files=[]
        generated_best_result_files=[]
        result_file = os.path.join(result_dir,
                               f'{algorithm_name}_dim_{dimension}_seed_{seed}.csv')
        best_solutions_result_file= os.path.join(result_dir,
                                   f'best_{algorithm_name}_dim_{dimension}_seed_{seed}.csv')
        all_populations_df = pd.DataFrame()
        best_solutions_df = pd.DataFrame()
        for problem_id in range(1,25):

            x_y_columns = [f'x_{i}' for i in range(0, dimension)] + ['y']

            for instance_id in range(1, instance_count+1):
                problem_name = f'bbob-{problem_id}-{instance_id}'
                problem = get_problem(problem_name, n_var=dimension)
                #cmaes = CMAES(x0=sampling, seed=seed)



                algorithm.termination = DefaultSingleObjectiveTermination()
                res = minimize(problem, termination=MaximumGenerationTermination(maximum_generations),
                               algorithm=algorithm, save_history=True,
                               seed=seed,
                               verbose=False)
                print(algorithm_name, problem_name)
                best_result = list(res.X) + list(res.F)
                best_result_data = tuple(best_result + [algorithm_name, problem_name, problem_id, instance_id])
                best_solutions_df=best_solutions_df.append(pd.DataFrame([best_result_data], columns = x_y_columns + ['algorithm_name','problem_name','problem_id','instance_id']))
                for iteration_index, iteration in enumerate(res.history):
                    pop_x = []
                    for population_individual in iteration.pop:
                        pop_x.append(list(population_individual.X) + list(population_individual.F))

                    population_df = pd.DataFrame(pop_x)
                    population_df['iteration'] = iteration_index
                    population_df['algorithm_name'] = algorithm_name
                    population_df['problem_id'] = problem_id
                    population_df['instance_id'] = instance_id
                    all_populations_df=all_populations_df.append(population_df)

                if instance_id%10==0:
                    all_populations_df.to_csv(result_file.replace('.csv', f'_{problem_id}_{instance_id}.csv'), compression='zip')
                    generated_trajectory_files+=[result_file.replace('.csv', f'_{problem_id}_{instance_id}.csv')]
                    best_solutions_df.to_csv(best_solutions_result_file.replace('.csv', f'_{problem_id}_{instance_id}.csv'), compression='zip')
                    generated_best_result_files+=[best_solutions_result_file.replace('.csv', f'_{problem_id}_{instance_id}.csv')]
                    all_populations_df = pd.DataFrame()
                    best_solutions_df = pd.DataFrame()



        #all_populations_df.to_csv(result_file)
        #best_solutions_df.to_csv(best_solutions_result_file)



        merge_generated_files(generated_trajectory_files, result_file)
        merge_generated_files(generated_best_result_files, best_solutions_result_file)

In [ ]:
if __name__ == '__main__':

    print ('Number of arguments:', len(sys.argv), 'arguments.')
    print('Argument List:', str(sys.argv))
    arguments = sys.argv
    seed, dimension = int(arguments[1]), int(arguments[2])
    run(seed,dimension, instance_count=100)